In [ ]:
# Under "Manual Changes" section
# 1. Select the correct file path for the file downloaded from databricks
# 2. Add the show name
# 3. Add the bonus names that will be analyzed
# 4. Add the Industry Average Ticket Price (manual calculation)
# 5. Add the points that the bonus awards

In [1]:
import pandas as pd
from pathlib import Path

# ========== MANUAL CHANGES ==========
file_path = ''
show_name = ''
bonus_names = ['', '']
IATP = 100
Points = '1000'
VIP_Only = 'No'

# ========== ALWAYS THE SAME ==========
first_sheet = 'Base Points'
second_sheet = 'Bonus Points'
column_header1 = 'show'
column_header2 = 'Offer Name'
transaction_date_column = 'Transaction Date'
performance_date_column = 'Performance Date'
ticket_quantity_column = 'Number of tickets'
ticket_price_column = 'Ticket Price'
points_column = 'Bonus Points Issued'
summary_title_1 = 'Bonus Sales'
summary_title_2 = show_name
Bonus_Type = 'main'

# Converting files into dataframes for analysis
df1 = pd.read_excel(file_path, sheet_name=first_sheet)
df2 = pd.read_excel(file_path, sheet_name=second_sheet)

# Finding the earliest purchase date
Sales_Range_Start_Date = df1[transaction_date_column].min()

# Finding the most recent purchase date
Sales_As_Of = df1[transaction_date_column].max()

# Finding total tickets based on show/bonus criteria
Total_Bonus_Tickets = df2[(df2[column_header1] == show_name) & (df2[column_header2].isin(bonus_names))][ticket_quantity_column].sum()

# Finding total dollars spent based on show/bonus criteria
Total_Bonus_Sales = df2[(df2[column_header1] == show_name) & (df2[column_header2].isin(bonus_names))][ticket_price_column].sum()

# Finding the ATP based on show/bonus criteria
ATP_Bonus_Eligible = Total_Bonus_Sales / Total_Bonus_Tickets if Total_Bonus_Tickets != 0 else None

# Finding the Industry Yield based on manual input above and caluclations so far
I_Yield = '{}%'.format(round((ATP_Bonus_Eligible / IATP)*100)) if IATP != 0 else 'N/A'

# Finding the cost of the bonus to the show
    #Additional date formatting code here for this function to work as well as to calculate days of the week later
df1[transaction_date_column] = pd.to_datetime(df1[transaction_date_column])
df2[transaction_date_column] = pd.to_datetime(df2[transaction_date_column])
df1[performance_date_column] = pd.to_datetime(df1[performance_date_column])
df2[performance_date_column] = pd.to_datetime(df2[performance_date_column])

# Rate determined based on main earning preference or earning in partner currency
def get_rate(bonus_names):
    if Bonus_Type in bonus_names:
        return 0.015
    else:
        return 0.03

filtered_df2 = df2[
    (df2[column_header1] == show_name) &
    (df2[column_header2].isin(bonus_names))
]
Bonus_Cost = sum(
    row[points_column] * get_rate(row[column_header2])
    for _, row in filtered_df2.iterrows()
)

# Finding total tickets based on show only
Total_Tickets = df1[df1[column_header1] == show_name][ticket_quantity_column].sum()

# Finding total dollars spent based on show only
Total_Sales = df1[df1[column_header1] == show_name][ticket_price_column].sum()

# Finding ATP based on show only
ATP_Total = Total_Sales / Total_Tickets if Total_Tickets != 0 else None

# Counting purchase and performance per days of the week
df1_filtered = df1[df1[column_header1] == show_name]
df2_filtered = df2[(df2[column_header1] == show_name) & (df2[column_header2].isin(bonus_names))]

day_counts_1 = df2_filtered[transaction_date_column].dt.day_name().value_counts()
day_counts_2 = df2_filtered[performance_date_column].dt.day_name().value_counts()
day_counts_3 = df1_filtered[transaction_date_column].dt.day_name().value_counts()
day_counts_4 = df1_filtered[performance_date_column].dt.day_name().value_counts()

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_counts_1 = day_counts_1.reindex(days, fill_value=0)
day_counts_2 = day_counts_2.reindex(days, fill_value=0)
day_counts_3 = day_counts_3.reindex(days, fill_value=0)
day_counts_4 = day_counts_4.reindex(days, fill_value=0)

# ========== SUMMARY TABLE BUILD ==========
summary_data = [
    ('Sales Range Start Date', Sales_Range_Start_Date),
    ('VIP Only?', VIP_Only),
    ('Points', Points),
    ('Sales As Of', Sales_As_Of),
    ('Total Bonus Sales', Total_Bonus_Sales),
    ('Total Bonus Tickets', Total_Bonus_Tickets),
    ('Average Ticket Price (bonus eligible tickets):', ATP_Bonus_Eligible),
    ('Industry Average Ticket Price', IATP),
    ('Yield over the Industry ticket price', I_Yield),
    ('Bonus Cost', Bonus_Cost),
]

# Add counts for each day of the week and separate bonus qualifying purchases from all ticket sales
summary_data += [('Transaction Dates for Bonus', '')]
summary_data += [(f'{day}', count) for day, count in day_counts_1.items()]
summary_data += [('Performance Dates for Bonus', '')]
summary_data += [(f'{day}', count) for day, count in day_counts_2.items()]
summary_data += [('Recap of Total Sales', '')]
summary_data += [('Total Sales', Total_Sales)]
summary_data += [('Total Tickets', Total_Tickets)]
summary_data += [('Avg Ticket Price', ATP_Total)]
summary_data += [('Transaction Dates for All Sales', '')]
summary_data += [(f'{day}', count) for day, count in day_counts_3.items()]
summary_data += [('Performance Dates for All Sales', '')]
summary_data += [(f'{day}', count) for day, count in day_counts_4.items()]

summary_df = pd.DataFrame(summary_data, columns=[summary_title_1, summary_title_2])

# ========== OUTPUT ==========
print(summary_df.to_string(index = False))

summary_df.to_excel(f'{show_name}_bonus_analysis.xlsx', index=False)


FileNotFoundError: [Errno 2] No such file or directory: ''